# **Importing Libraries**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# **Importing Dataset**

In [4]:
def import_dataset(path: str ,nrows: int | None = None) -> pd.DataFrame:
    train_df = pd.read_csv(path, nrows=nrows)

    machine_area_df = pd.read_csv("dataset/Machine-Area.csv")
    machine_list_df = pd.read_csv("dataset/Machine-List.csv")
    area_list_df = pd.read_csv("dataset/Area-List.csv")

    machine_area_df['Last Maintenance'] = pd.to_datetime(machine_area_df['Last Maintenance'])
    machine_area_filtered_df = machine_area_df.sort_values(by=['ID_Area', 'ID_Mesin', 'Last Maintenance'], ascending=[True, True, False])
    machine_area_filtered_df = machine_area_filtered_df.drop_duplicates(subset=['ID_Area', 'ID_Mesin'], keep='first')

    combined_df = pd.merge(train_df, area_list_df, on='ID_Area', how='left')
    combined_df = pd.merge(train_df, machine_list_df, left_on='Machine', right_on='ID_Mesin', how='left')
    combined_df = pd.merge(train_df, machine_area_filtered_df, left_on=['Machine', 'ID_Area'], right_on=['ID_Mesin', 'ID_Area'], how='left')

    return combined_df
    

In [11]:
train_df = import_dataset(
    path="dataset/train.csv", 
    nrows=1_000_000)

# **Data Cleaning**

## Handling Missing Data

## Dealing with Outliers

## Remove Duplicates

## Feature Engineering

# **Data Preprocessing**

## Feature Scaling

## Feature Encoding

## Handling Imbalanced Dataset

# **Modelling & Validation**

# **Result & Analysis**